In [ ]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import scipy.stats
%matplotlib inline

## ANÁLISE CHI-SQUARE: IMPACTO DO CONTEÚDO POLÍTICO POR FAIXA ETÁRIA

In [ ]:
df = pandas.read_csv('dataset.csv')
print(f"Dataset original: {len(df)} registros")

### Limpando os dados (pra corrigir alguns valores do dataset)

In [ ]:
df_clean = df[df['Age'] >= 18].copy()
print(f"Após limpeza (idade >= 18): {len(df_clean)} registros")

## Listagem do personal impact pra análise

In [ ]:
print(df_clean['Personal Impact'].value_counts())

## Criando grupos de idade

In [ ]:
def categorizar_idade(idade):
    if 18 <= idade <= 30:
        return 1  # Jovens
    elif 31 <= idade <= 50:
        return 2  # Adultos  
    elif idade >= 51:
        return 3  # Idosos
        
df_clean['FAIXAETARIA'] = df_clean['Age'].apply(categorizar_idade)

## Criação de váriavel pra facilitar a análise (sim ou não)

In [ ]:
def categorizar_impacto(impacto):
    if impacto in ['Strong impact', 'Moderate impact']:
        return 1  # Com impacto
    elif impacto == 'No impact':
        return 0  # Sem impacto

df_clean['IMPACTO'] = df_clean['Personal Impact'].apply(categorizar_impacto)

## Limpando valores nulos do dataset pra facilitar a análise

In [ ]:
sub2 = df_clean.dropna(subset=['FAIXAETARIA', 'IMPACTO']).copy()
print(f"Dataset final: {len(sub2)} registros")

## Tabela de contingência

In [ ]:
ct1 = pandas.crosstab(sub2['IMPACTO'], sub2['FAIXAETARIA'])
print("\nTabela de Contingência:")
print(ct1)

## Soma todos os valores por coluna e pego o porcentual por coluna

In [ ]:
colsum = ct1.sum(axis=0)
colpct = ct1/colsum

print("\nPercentuais por coluna (faixa etária):")
print(colpct)

print('\nValor de Qui-quadrado, Valor p, valores esperados')
cs1 = scipy.stats.chi2_contingency(ct1)
print(cs1)

In [ ]:
sub2["FAIXAETARIA"] = sub2["FAIXAETARIA"].astype('category')

sub2['IMPACTO'] = pandas.to_numeric(sub2['IMPACTO'], errors='coerce')

## Gráfico

In [ ]:
seaborn.catplot(x="FAIXAETARIA", y="IMPACTO", data=sub2, kind="bar", ci=None)
plt.xlabel('Faixa Etária (1=Jovens, 2=Adultos, 3=Idosos)')
plt.ylabel('Proporção de Percepção de Impacto')
plt.title('Percepção de Impacto por Faixa Etária')
plt.show()

## Jovens vs Idosos

In [ ]:
recode2 = {1: 1, 3: 3}  # 1=Jovens, 3=Idosos
sub2['COMP1v3'] = sub2['FAIXAETARIA'].map(recode2)

ct2 = pandas.crosstab(sub2['IMPACTO'], sub2['COMP1v3'])
print("\n" + "="*50)
print("COMPARAÇÃO JOVENS vs IDOSOS")
print("="*50)
print(ct2)

## Percentuais por coluna

In [ ]:
colsum = ct2.sum(axis=0)
colpct = ct2/colsum
print("\nPercentuais por coluna:")
print(colpct)

print('\nValor de Qui-quadrado, Valor p, Valores esperados')
cs2 = scipy.stats.chi2_contingency(ct2)
print(cs2)

## Interpretação dos resultados

In [ ]:
print("\n" + "="*50)
print("INTERPRETAÇÃO DOS RESULTADOS")
print("="*50)

chi2_stat, p_value, dof, expected = cs1
print(f"Teste geral (todas as faixas etárias):")
print(f"Chi-square: {chi2_stat:.4f}")
print(f"P-valor: {p_value:.6f}")
print(f"Significativo (p < 0.05): {'SIM' if p_value < 0.05 else 'NÃO'}")

chi2_stat2, p_value2, dof2, expected2 = cs2
print(f"\nTeste específico (Jovens vs Idosos):")
print(f"Chi-square: {chi2_stat2:.4f}")
print(f"P-valor: {p_value2:.6f}")
print(f"Significativo (p < 0.05): {'SIM' if p_value2 < 0.05 else 'NÃO'}")


## Descritivo em %

In [ ]:
print(f"\n" + "="*50)
print("ESTATÍSTICAS DESCRITIVAS")
print("="*50)

faixas = {1: 'Jovens (18-30)', 2: 'Adultos (31-50)', 3: 'Idosos (51+)'}
for codigo, nome in faixas.items():
    subset = sub2[sub2['FAIXAETARIA'] == codigo]
    com_impacto = len(subset[subset['IMPACTO'] == 1])
    total = len(subset)
    pct = (com_impacto / total * 100) if total > 0 else 0
    print(f"{nome}: {com_impacto}/{total} ({pct:.1f}%) percebem impacto")